In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import datetime as dt

# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [ ]:
#Read csv files

In [ ]:
# Read files into pandas dataframe


In [ ]:
# Drop unnecessary columns

# Add id to dataworld dataframe

# filter out necessary data: 1. non-american made movies, 2. Release date is before 1960


In [ ]:
# Summaries
# Genre counts

# Gross Revenue by Year

# Average ratings by revenue/budget bins

In [ ]:
# load files into sql database
# Create Engine and Pass in MySQL Connection
#engine = create_engine("link to be added")